# A simple test
This will be a test to extract data from one IMU (through the Arduino) and format the data into a database to be imported to a matplotlib.

Based on the previously written code (with the visualisation of the MicroBit) but having the vector points as singular points. 

## Single IMU Test


### Arduino Code
Use this site to rewrite the Arduino code that I have to work more effectively: https://howtomechatronics.com/tutorials/arduino/arduino-and-mpu6050-accelerometer-and-gyroscope-tutorial/



### Python Code: 1
This is what has extracted the serial data from the basic arduino code (that I actually wrote).

It became increasingly messy and difficult to troubleshoot; I am unable to get the matplotlib to run simultaneously and will rewrite this to find the specific error. 

In [ ]:
import serial.tools.list_ports
import string
import serial
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()

portList = []

for port in ports:
    portList.append(str(port))
    print(str(port))
    
val = input("Select port: COM")

portVar = None
program = True




"""# write a class for accelerometer visualisation
# the goal currently is just to get the data updating live
# without having an inputted time delay (i.e. at the rate of seeing the data)
# and hoping the laptop doesn't undergo cardiac arrest
def vector_acceleration(acceleration_list):
    
    # return 
    return"""
plt.ion()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')

while program:
    for x in range(len(portList)):
        if portList[x].startswith("COM" + str(val)):
            portVar = "COM" + str(val)
            print(portList[x])
            
    if portVar is None:
        print("There are no open ports")
        program = False
        break
            
    serialInst.baudrate = 9600
    serialInst.port = portVar
    serialInst.open()

    while True:
        try: 
            if serialInst.in_waiting: # if there is data in the buffer
                decoded_packet = serialInst.readline().decode('utf-8')
                line = decoded_packet.lstrip() # removing whitespaces at the beginning
                
                
                if line and (line[0].isdigit() or line[0] == '-'): # if line is not empty and {...}
                    acc_tilt = line.split("|")
                    
                    # acceleration
                    acceleration = acc_tilt[0].split(" ")
                    acc_list = str(acceleration[0]).split("\t")
                    acc_vector = []
                    
                    if len(acc_list) >= 3:
                        x_acc = acc_list[0]
                        y_acc = acc_list[1]
                        z_acc = acc_list[2]
                        acc_vector.append((x_acc, y_acc, z_acc))
                        
                        
                        plt.clf()
                        ax.plot3D(float(x_acc), float(y_acc), float(z_acc))
                        ax.set_proj_type('ortho')
                        ax.set_xlabel('X Axis')
                        ax.set_ylabel('Y Axis')
                        ax.set_zlabel('Z Axis')
                        ax.set_title('Acceleration vector mapping with 1 IMU')
                        plt.pause(1)
                        
                        print(float(x_acc))
                    
                    # tilt
                    """tilt_very_unclean = acc_tilt[1].strip()
                    tilt_list = tilt_very_unclean.split("\t")
                    tilt_01 = tilt_list[0] # not sure what this is actually measuring
                    tilt_02 = tilt_list[1] # not sure what this is actually measuring""" 
                    
                    
                    
        except Exception as e:
           print(f"{e} error.\nTry again.")
                
      
            
    # decode this data further so that it can be used with the matplotlib code
    # make it easier to stop and start the flow of data through combinations of key presses (for example)
    # understand how to send transformed data across back from the serial  
       # in a different way/different channel since I want this to pass through an (e.g.) ESP to send data to haptic motors
       # need to understand how to communicate between multiple pieces of code automatically
       # to display this clearly"""



### Python Code: 2
A reattempt that focuses on the plotting of data using more efficient modules

In [ ]:
import serial.tools.list_ports
import string
import serial
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time

# reading the serial data

try:
    ports = serial.tools.list_ports.comports()
    serialInst = serial.Serial()

    portList = [i for i in ports]
    if len(portList) == 1:
        portVar = ports[0].device
        print("Connection successful")

    serialInst.baudrate = 9600
    serialInst.port = portVar
    print(portVar)
    serialInst.open()
    
except Exception as e:
    print(f"{e}")
    
# decode serial data

# write class + function to plot acceleration as a quiver
class SensorData:
    def __init__(self, serial_path):
        
    def accelerationData(self):
        return self
        
    def gyroscopeData(self):
        return self
        
    def fusionData(self):	# a mix between both acceleration and gyroscope
        return self         # to control the amount of drift (and overall
                            # accuracy) of the sensors
    
class SensorVisualiser:
    def __init__(self, ...):
        
    def forward_kinematics(self)
        # first test will be basic vector lines joined up
        # later tests will embed the cubic spline interpretation
            # to produce a more accurate model
        return self
    
    def display_tilt_angles(self):
        return self

# call function


# show graph
def main():
    # ... {processes and if statements} ...
    plt.show()
    
main()



### Simplified Python
##### Next Steps
- How would I want this displayed as a final digital twin? 
    - Do I want the spine model to show the trace of how it has moved? Will its display movement be interpolated and smoothed? 
    - Experiment with different visualisations (quiver vector, displaying integral and vector coordinates live in matplotlib)
- Reading more than one IMU at once (hopefully 3 IMUs)
    - From that, I can focus on the spline calculations and forward kinematics
- Develop the error checking for this (for all use cases (relatively minor))
- Displaying (in the matplotlib code) the acceleration (or more generally just data that a user could care about)
    - https://stackoverflow.com/questions/43373569/how-to-display-a-text-with-matplotlib {a useful resource that inspired its application}

In [ ]:
import serial.tools.list_ports
import string
import serial
import matplotlib.offsetbox as osb
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
from serial import SerialException


def animate(i, dataList, ser):
    
    try:
        IMU_data_string = ser.readline().decode('utf-8').lstrip()

        line = IMU_data_string
        
        if line and (line[0].isdigit() or line[0] == '-'):
            
            IMU_split_data = line.split("|") # separates acceleration to tilt data
            IMU_acc = str(IMU_split_data[0]).split("\t") # only reads acceleration
                        
            # ensure there are three values to read
            if len(IMU_acc) >= 3:
                x_acc = float(IMU_acc[0])
                y_acc = float(IMU_acc[1])
                z_acc = float(IMU_acc[2])
                
                # store the data points
                dataList.append([x_acc, y_acc, z_acc])
                dataList = dataList[-50:] # fix list size
                
                ax.clear()
                ax.scatter(x_acc, y_acc, z_acc)
                
                ax.set_xlabel('X Acceleration')
                ax.set_ylabel('Y Acceleration')
                ax.set_zlabel('Z Acceleration')

                ax.set_title("Vector Position of Acceleration")
                
                return x_acc, y_acc, z_acc
                
                
    except:
        pass

    
dataList = []

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection = '3d')

try:
    ser = serial.Serial("COM4", 9600)
    time.sleep(2)
    print("Connected successfully")
except SerialException as e:
    print(f"{e}.\n\nAn error has occurred, please try again later.")
    exit(1) # exit code that means abandoning program since there were errors
            # exit(0) means a clean exit

"""i = [i for i in range(100000)] 
x_acc, y_acc, z_acc = animate(i, dataList, ser)"""
    

ani = animation.FuncAnimation(fig, animate, frames=100, fargs=(dataList, ser), interval=100, blit=False, cache_frame_data=False) # find out what "blit" and "cache_frame_data" means

"""text = osb.AnchoredText("Acceleration: {},{},{}".format(x_acc, y_acc, z_acc))
plt.gca.add_artist(text) # gca means "get current axes" """
plt.show()
ser.close()
    